# numpy 사용

In [1]:
import numpy as np

batchSize, inputLayer, hiddenLayer, outputLayer = 64, 1000, 100, 10

x = np.random.randn(batchSize, inputLayer)
y = np.random.randn(batchSize, outputLayer)

w1 = np.random.randn(inputLayer, hiddenLayer)
w2 = np.random.randn(hiddenLayer, outputLayer)

lr = 1e-6

for epoch in range(500):
    # 순전파 단계: 예측값 y 계산
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    #  Loss를 계산하고 출력
    loss = np.square(y_pred - y).sum()
    print(epoch, loss)

    # 손실에 따른 w1, w2의 변화도를 계산하고 역전파
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    w1 -= lr * grad_w1
    w2 -= lr * grad_w2

0 33172623.66426563
1 35121999.28960111
2 42299695.820782706
3 46474377.599419236
4 39164549.208869286
5 23495372.248044875
6 10405012.095174978
7 4325365.418492799
8 2140995.83325008
9 1362278.3865940957
10 1018862.1698861377
11 820943.6589290784
12 682412.2627981608
13 575525.084357893
14 489728.8914945789
15 419480.55425753084
16 361233.6374714101
17 312541.1296923335
18 271574.04019493714
19 236963.3137993855
20 207512.5876530419
21 182378.04118084238
22 160814.25275062938
23 142234.1125555866
24 126154.65639385625
25 112173.9938708458
26 99994.92136965325
27 89346.03080265477
28 80009.47235520533
29 71795.23877049563
30 64552.81006901731
31 58154.02384320396
32 52487.40357620381
33 47453.53729460374
34 42975.462749358485
35 38978.53958534695
36 35404.28942380005
37 32204.536070441438
38 29335.34082865022
39 26754.48170465466
40 24429.194704676316
41 22331.469398212088
42 20437.60116522782
43 18722.95565925293
44 17170.199964606425
45 15761.7502677457
46 14482.583345488776
47 13319

# pytorch 사용

In [2]:
import torch
import torchvision


In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [4]:
dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")
print(device)

batchSize, inputLayer, hiddenLayer, outputLayer = 64, 1000, 100, 10

lr = 1e-6
x = torch.randn(batchSize, inputLayer, device=device, dtype=dtype)
y = torch.randn(batchSize, outputLayer, device=device, dtype=dtype)

w1 = torch.randn(inputLayer, hiddenLayer, device=device, dtype=dtype)
w2 = torch.randn(hiddenLayer, outputLayer, device=device, dtype=dtype)

lr = 1e-6
for epoch in range(500):
    # 순전파 단계: 예측값 y를 계산
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)
    
    # 손실을 계산하고 출력합니다.
    loss = (y_pred - y).pow(2).sum().item()
    if epoch % 100 == 99: print(epoch, loss)

    # 손실에 따른 w1, w2의 변화도를 계산하고 역전파합니다. 
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # 경사하강법을 사용하여 가중치를 갱신합니다.
    w1 -= lr * grad_w1
    w2 -= lr * grad_w2

cpu
99 461.3334655761719
199 1.8259186744689941
299 0.010903755202889442
399 0.0002197134162997827
499 3.5271506931167096e-05


# Autograd

Autograd를 사용하여 신경망에서 역전파 단계 연산을 자동화 할 수 있습니다. Autograd를 사용할 때, 신경망의 순전파 단계는 연산 그래프를 정의하게 됩니다. 이 그래프의 노드는 Tensor, 엣지는 입력 Tensor로부터 출력 Tensor를 만들어내는 함수가 됩니다. 이 그래프를 통해 역전파를 하게되면 변화도를 쉽게 계산할 수 있습니다.

In [7]:
dtype = torch.float
device = torch.device("cpu")

batchSize, inputLayer, hiddenLayer, outputLayer = 64, 1000, 100, 10

x = torch.randn(batchSize, inputLayer, device=device, dtype=dtype)
y = torch.randn(batchSize, outputLayer, device=device, dtype=dtype)

w1 = torch.randn(inputLayer, hiddenLayer, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(hiddenLayer, outputLayer, device=device, dtype=dtype, requires_grad=True)

lr = 1e-6

for epoch in range(500):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    loss = (y_pred - y).pow(2).sum()
    if epoch % 100 == 99: print(epoch, loss.item())

    loss.backward()

    with torch.no_grad():
        w1 -= lr * w1.grad
        w2 -= lr * w2.grad

        w1.grad.zero_()
        w2.grad.zero_()

99 647.7728271484375
199 3.8132331371307373
299 0.029090795665979385
399 0.0004981890087947249
499 6.34252282907255e-05


# 새로운 Autograd 함수 정의

내부적으로, autograd의 기본 연산자는 실제로 Tensor를 조작하는 2개의 함수입니다. forward 함수는 입력 Tensor로부터 출력 Tensor를 계산합니다. backward 함수는 어떤 스칼라 값에 대한 출력 Tensor의 변화도를 전달받고, 동일한 스칼라 값에 대한 입력 Tensor의 변화도를 계산합니다.

PyTorch에서 torch.autograd.Function의 서브클래스를 정의하고 forward와 backward를 구현함으로써 사용자 정의 autograd 연산자를 손쉽게 정의할 수 있습니다. 그 후, 인스턴스를 생성하고 이를 함수처럼 호출하여 입력 데이터를 갖는 Tensor를 전달하는 식으로 새로운 autograd 연산자를 사용할 수 있습니다.

In [8]:
class MyReLU(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input_):
        ctx.save_for_backward(input_)
        return input_.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        input_, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input_ < 0] = 0

        return grad_input